In [ ]:
#| default_exp yolo

In [ ]:
#| export
from ultralytics import YOLO
import json
import os

In [ ]:
import textwrap
import json
import openai
from llmcam.fn_to_fc import *

In [ ]:
#| export

def detect_objects(
        image_path: str, # Path/URL of image
        conf: float=0.25 # Confidence threshold
    ) -> str: # JSON format of detection results
    """Detect object in the input image."""
    model = YOLO('yolov8s.pt')
    
    result = model(image_path, conf=conf, exist_ok=True)[0]
    # result.show()
    save_dir = os.getenv("LLMCAM_DATA", "../data")
    output_path = image_path.split("/")[-1]
    result.save(filename=f"{save_dir}/detection_{output_path}")
    dict = {}
    for c in result.boxes.cls:
        dict[model.names[int(c)]] = dict.get(model.names[int(c)], 0) + 1
    return json.dumps(dict)

In [ ]:
#| eval: false

detect_objects("../data/cap_2024.11.25_10:59:00_SantaClausVillage.jpg")


image 1/1 /home/abc/llmcam/nbs/../data/cap_2024.11.25_10:59:00_SantaClausVillage.jpg: 384x640 27 persons, 1 snowboard, 15.6ms
Speed: 1.7ms preprocess, 15.6ms inference, 4.4ms postprocess per image at shape (1, 3, 384, 640)


'{"person": 27, "snowboard": 1}'

In [ ]:
YTLiveTools = [tool_schema(fn) for fn in (capture_youtube_live_frame_and_save, ask_gpt4v_about_image_file, detect_objects)]

In [ ]:
def exec_func(name, tools=[], **kwargs):
    res = globals().get(name)(**kwargs)
    return res

In [ ]:
#| eval: false
messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to assist the user."),
    ("user", "Hi, can you capture YouTube Live?"),
    ("user", "You can use the default link."),
    ("user", "Can you extract information from this file?"),
    ("user", "Use your new tool to detect."),
])
complete(messages, YTLiveTools, exec_func)
print_msgs(messages)